In [0]:
!pip install tensorflow-gpu

     |████████████████████████████████| 421.8MB 35kB/s 
     |████████████████████████████████| 450kB 45.9MB/s 
     |████████████████████████████████| 3.8MB 44.0MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.1.0 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.1.0 which is incompatible.
  Found existing installation: tensorflow-estimator 1.15.1
    Uninstalling tensorflow-estimator-1.15.1:
      Successfully uninstalled tensorflow-estimator-1.15.1
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0


In [0]:
import tensorflow
tensorflow.__version__

'2.1.0'

In [0]:
from PIL import Image
import os
from tqdm import tqdm_notebook
import numpy as np
from sklearn.feature_extraction.image import extract_patches_2d
import matplotlib.pyplot as plt 
import cv2
import pywt
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split
import pywt
from sklearn.utils import shuffle
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
DATA_PATH = "/content/drive/My Drive/Casia/Casia2/"


In [0]:
X_raw = np.load(DATA_PATH+"X_patch_train.npy")
Y_raw = np.load(DATA_PATH+"Y_patch_train.npy")

In [0]:
# Create balance pos-neg dataset
thresh=0.4
pos_X = X_raw[Y_raw>thresh]
neg_X = X_raw[Y_raw==0]
pos_Y = Y_raw[Y_raw>thresh]
num_data = pos_X.shape[0]
neg_Y = Y_raw[Y_raw==0]
neg_X = shuffle(neg_X)[:num_data]

In [0]:
num_data

7967

In [0]:
num_data

38619

In [0]:
X = np.concatenate((pos_X,neg_X),axis=0)
Y = np.concatenate((np.ones(num_data),np.zeros(num_data)),axis=0)
np.save(DATA_PATH + "Y_wt_train.npy",Y)

In [0]:
thresh=0.4
pos_X = X_raw[Y_raw>thresh]
num_data = pos_X.shape[0]
Y_wt = np.load(DATA_PATH + "Y_wt_train.npy")
Y = np.concatenate((np.ones(num_data),np.zeros(num_data)),axis=0)


In [0]:
def mu_std_sum(matrix):
  return [np.mean(matrix),np.std(matrix),np.sum(matrix)]

def extract_features(image):

  # Transpose to iterate through channels
  image = np.transpose(image,(2,0,1))
  vector = []

  for channel in image[1:]: # Remove Y Channel

    # iterate through db1->db5
    for i in range(1,6):
      coeffs = pywt.wavedec2(channel,"db"+str(i),level=3)

      # Append first mat mu_std_sum
      vector += mu_std_sum(coeffs[0])

      for dec in coeffs[1:]:
        vector += mu_std_sum(dec[0])
        vector += mu_std_sum(dec[1])
        vector += mu_std_sum(dec[2])

  return vector
      


In [0]:
X_wt=[]
for item in tqdm_notebook(X):
  X_wt.append(extract_features(item))
X_wt=np.array(X_wt)

/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))


In [0]:
np.save(DATA_PATH+"X_wt_train.npy",X_wt)
#X_wt = np.load(DATA_PATH + "X_wt_train.npy")

In [0]:
X_wt = np.load(DATA_PATH+"X_wt_train.npy")

In [0]:
X_wt.shape

(15934, 300)

In [0]:
def recall_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

def precision_m(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))


In [0]:
import sklearn
scaler_obj = sklearn.preprocessing.MinMaxScaler()
X_wt_scaled = scaler_obj.fit_transform(X_wt)

In [0]:
import sklearn
sklearn.preprocessing

<module 'sklearn.preprocessing' from '/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/__init__.py'>

In [0]:
indices = np.arange(Y.shape[0])
np.random.shuffle(indices)
X_wt_scaled_ = X_wt_scaled[indices]
Y_ = Y[indices]

In [0]:
X_wt_scaled_

array([[0.4823891 , 0.59330357, 0.4823891 , ..., 0.57814416, 0.07173087,
        0.57814416],
       [0.46917369, 0.04722534, 0.46917369, ..., 0.58925985, 0.00874757,
        0.58925985],
       [0.33939824, 0.11420464, 0.33939824, ..., 0.58375864, 0.02441784,
        0.58375864],
       ...,
       [0.41156433, 0.02241958, 0.41156433, ..., 0.59689137, 0.01795309,
        0.59689137],
       [0.42416698, 0.04141214, 0.42416698, ..., 0.59398946, 0.04871889,
        0.59398946],
       [0.4079167 , 0.12680828, 0.4079167 , ..., 0.58534565, 0.02193269,
        0.58534565]])

In [0]:
from tensorflow import keras
from tensorflow.keras import optimizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D,Dropout
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard,ModelCheckpoint
from tensorflow.keras.metrics import BinaryAccuracy

cp = ModelCheckpoint(DATA_PATH + "weights.{epoch:02d}-{val_accuracy:.4f}.h5",
                     monitor='val_accuracy', 
                     verbose=1, 
                     save_best_only=True,
                     mode='max',
                     save_weights_only=True)

def build_model():

    model = Sequential()

    model.add(Dense(450, activation = 'relu', kernel_initializer='he_normal'))

    model.add(Dense(300, activation = 'relu', kernel_initializer='he_normal'))

    model.add(Dense(300, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))

    model.add(Dense(200, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))

    model.add(Dense(150, activation = 'relu', kernel_initializer='he_normal'))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation = 'relu', kernel_initializer='he_normal'))
    
    model.add(Dropout(0.5))

    model.add(Dense(1, activation = 'sigmoid', kernel_initializer='he_normal'))
    model.compile(loss='binary_crossentropy', 
                  optimizer='adam', 
                  metrics=["accuracy"])
    return model

In [0]:
model = build_model()
BATCH_SIZE = 256
model.fit(X_wt_scaled_,Y_,
          batch_size=BATCH_SIZE,
          epochs=1000,
          validation_split=0.22,
          callbacks=[cp])

11264/12428 [==========================>...] - ETA: 0s - loss: 0.3022 - accuracy: 0.8665
Epoch 00226: val_accuracy did not improve from 0.82972
12428/12428 [==============================] - 0s 18us/sample - loss: 0.3005 - accuracy: 0.8673 - val_loss: 0.5058 - val_accuracy: 0.8046
Epoch 227/1000
11776/12428 [===========================>..] - ETA: 0s - loss: 0.2803 - accuracy: 0.8748
Epoch 00227: val_accuracy did not improve from 0.82972
12428/12428 [==============================] - 0s 17us/sample - loss: 0.2783 - accuracy: 0.8765 - val_loss: 0.5393 - val_accuracy: 0.8135
Epoch 228/1000
11264/12428 [==========================>...] - ETA: 0s - loss: 0.3005 - accuracy: 0.8675
Epoch 00228: val_accuracy did not improve from 0.82972
12428/12428 [==============================] - 0s 18us/sample - loss: 0.3026 - accuracy: 0.8664 - val_loss: 0.4553 - val_accuracy: 0.8189
Epoch 229/1000
11520/12428 [==========================>...] - ETA: 0s - loss: 0.2876 - accuracy: 0.8727
Epoch 00229: val_acc

KeyError: ignored

In [0]:
model.load_weights("/content/drive/My Drive/Casia/Casia2/weights.186-0.8297.h5")

In [0]:
X_au_test = np.load(DATA_PATH + "X_au_test.npy")
X_tp_test = np.load(DATA_PATH + "X_tp_test.npy")
X_au_train = np.load(DATA_PATH + "X_au_train.npy")
X_tp_train = np.load(DATA_PATH + "X_tp_train.npy")

In [0]:
def img2batch(arr,shape=(32,32)):

  nrows, ncols = shape
  batches = []
  dim = len(arr.shape)
  if dim == 3:
    h, w,_ = arr.shape
  elif dim ==2:
    h, w = arr.shape

  assert h % nrows == 0, "{} rows is not evenly divisble by {}".format(h, nrows)
  assert w % ncols == 0, "{} cols is not evenly divisble by {}".format(w, ncols)


  for r in range(h//nrows):
    for c in range(w//ncols):
      from_r = r*nrows
      to_r = (r+1)*nrows
      from_c = c*ncols
      to_c =(c+1)*ncols
      if dim == 3:
        batches.append(arr[from_r:to_r,from_c:to_c,:])
      elif dim==2:
        batches.append(arr[from_r:to_r,from_c:to_c])

  return batches

In [0]:
idx = 20

batches = img2batch(X_au_test[idx])
label = []
for batch in batches:
  feature = np.array(extract_features(batch))
  feature = np.expand_dims(feature,0)
  feature = scaler_obj.transform(feature)
  label.append(model.predict(feature)[0][0])

/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/dist-packages/pywt/_multilevel.py:45: UserWarning: Level value of 3 is too high: all coefficients will experience boundary effects.
  "boundary effects.").format(level))
/usr/local/lib/python3.6/

In [0]:
thresh = 0.5
label = (np.array(label)>thresh)*1

In [0]:
import collections
collections.Counter(label)

Counter({0: 96})

In [0]:
def pkl_save(obj,path):
  import pickle
  with open(path, 'wb') as fp:
    pickle.dump(obj, fp)
def pkl_load(path):
  import pickle
  with open(path, 'rb') as fp:
    obj = pickle.load(fp)
  return obj

In [0]:
pkl_save(scaler_obj,"scaler_obj.pkl")

In [0]:
hey = pkl_load(DATA_PATH+ "scaler_obj.pkl")

In [0]:
max

<function max>

In [0]:
scaler_obj.min_.shape

(300,)

In [0]:
scaler_obj.data_max_.shape

(300,)

In [0]:
min_ = np.expand_dims(scaler_obj.data_min_,0)
max_ = np.expand_dims(scaler_obj.data_max_,0)
np.save("min_data.npy",min_)
np.save("max_data.npy",max_)

In [0]:
X_wt_scaled_2 = (X_wt - min_)/(max_-min_)

In [0]:
X_wt_scaled

array([[0.45507804, 0.16422949, 0.45507804, ..., 0.60696169, 0.01946686,
        0.60696169],
       [0.47422984, 0.13922344, 0.47422984, ..., 0.61231943, 0.0293241 ,
        0.61231943],
       [0.42906969, 0.04059922, 0.42906969, ..., 0.61734923, 0.22071207,
        0.61734923],
       ...,
       [0.47593689, 0.05945733, 0.47593689, ..., 0.6150931 , 0.02120154,
        0.6150931 ],
       [0.41912799, 0.04276628, 0.41912799, ..., 0.62326178, 0.01982803,
        0.62326178],
       [0.41999002, 0.04875165, 0.41999002, ..., 0.60112664, 0.04327358,
        0.60112664]])

In [0]:
X_wt_scaled_2

array([[0.45507804, 0.16422949, 0.45507804, ..., 0.60696169, 0.01946686,
        0.60696169],
       [0.47422984, 0.13922344, 0.47422984, ..., 0.61231943, 0.0293241 ,
        0.61231943],
       [0.42906969, 0.04059922, 0.42906969, ..., 0.61734923, 0.22071207,
        0.61734923],
       ...,
       [0.47593689, 0.05945733, 0.47593689, ..., 0.6150931 , 0.02120154,
        0.6150931 ],
       [0.41912799, 0.04276628, 0.41912799, ..., 0.62326178, 0.01982803,
        0.62326178],
       [0.41999002, 0.04875165, 0.41999002, ..., 0.60112664, 0.04327358,
        0.60112664]])